## Libraries

In [1]:
import os

import numpy as np
import pandas as pd
from pprint import pprint

from pathlib import Path

import awpy.data as awpydata
from awpy.data import MAP_DATA


from awpy import DemoParser
from awpy.analytics.stats import (
    player_stats
)

from awpy.analytics import states
from awpy.analytics import nav
from awpy.data import (NAV,NAV_CSV)
import plotly.graph_objects as go

import matplotlib.pyplot as plt
from awpy.visualization.plot import (
    plot_map, 
    plot_round, 
    plot_positions, 
    plot_nades,
    position_transform)

import plotly.tools as tls
from plotly.offline import download_plotlyjs, init_notebook_mode, iplot
import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff
from imageio.v3 import imread

In [2]:
data_path = Path("/Users/iqbal/Documents/mirandalabs/datasets/BLAST-Premier-Fall-Series-2020-og-vs-natus-vincere-bo3")

In [3]:
demo_parser = DemoParser(
    demofile = f"{data_path}/og-vs-natus-vincere-m1-dust2.dem",
    outpath= Path.cwd(),
    demo_id = "OG-NaVi-BLAST2020", 
    parse_rate=128, 
    trade_time=5, 
    buy_style="hltv"
)


# Parse the demofile, output results to dictionary
data = demo_parser.parse()


In [4]:
player_stats_json = player_stats(data["gameRounds"])

In [5]:
cats = [
        "kills",
        "assists",
        'totalShots',
        "tradeKills"]

In [6]:
df= pd.DataFrame(player_stats_json).T

In [8]:
df

,steamID,playerName,teamName,isBot,totalRounds,kills,deaths,kdr,assists,tradeKills,teamKills,suicides,flashAssists,totalDamageGiven,totalDamageTaken,totalTeamDamageGiven,adr,totalShots,shotsHit,accuracy,rating,kast,hs,hsPercent,firstKills,firstDeaths,utilityDamage,smokesThrown,flashesThrown,heThrown,fireThrown,enemiesFlashed,teammatesFlashed,blindTime,plants,defuses
76561198016432560 - mantuu,76561198016432560,mantuu,OGEsports,False,25,24,10,2.4,1,7,0,0,2,2079,1259,0,83.2,235,49,0.21,1.59,88.0,8,0.33,5,1,60,18,28,6,8,19,49,52.4,2,0
76561198013243326 - Aleksib,76561198013243326,Aleksib,OGEsports,False,25,17,16,1.06,4,2,0,0,3,2027,1990,0,81.1,437,53,0.12,1.2,84.0,10,0.59,4,2,270,17,30,10,14,24,33,63.88,2,0
76561197960710573 - NBK-,76561197960710573,NBK-,OGEsports,False,25,22,16,1.38,7,6,0,0,1,2716,1724,0,108.6,562,89,0.16,1.44,80.0,9,0.41,2,3,51,14,17,8,9,17,16,43.39,1,0
76561198114929868 - valde,76561198114929868,valde,OGEsports,False,25,12,17,0.71,2,4,0,0,1,1489,1737,3,59.6,409,52,0.13,0.78,64.0,5,0.42,1,2,177,16,28,8,11,27,36,62.86,1,0
76561198037812456 - ISSAA,76561198037812456,ISSAA,OGEsports,False,25,13,15,0.87,3,2,0,0,0,1322,1886,40,52.9,400,38,0.1,0.93,76.0,8,0.62,2,3,117,14,18,7,10,14,18,20.86,2,0
76561198121220486 - Perfecto,76561198121220486,Perfecto,Natus Vincere,NaN,25,11,17,0.65,5,2,0,0,3,1383,1757,1,55.3,416,56,0.13,0.83,76.0,4,0.36,1,2,32,16,24,7,11,30,24,72.27,1,0
76561198034202275 - s1mple,76561198034202275,s1mple,Natus Vincere,NaN,25,21,16,1.31,1,3,0,0,0,2031,2019,32,81.2,214,43,0.2,1.21,68.0,6,0.29,2,3,182,12,21,3,4,14,20,39.68,2,0
76561198116523276 - flamie,76561198116523276,flamie,Natus Vincere,NaN,25,15,19,0.79,3,5,0,0,3,1558,2026,0,62.3,241,37,0.15,0.94,76.0,11,0.73,2,1,138,11,28,3,12,43,25,89.57,2,0
76561198044045107 - electronic,76561198044045107,electronic,Natus Vincere,NaN,25,15,19,0.79,3,5,0,1,0,1748,2159,0,69.9,308,63,0.2,0.85,60.0,7,0.47,3,3,48,14,23,4,8,25,28,57.34,1,1
76561198146207066 - Boombl4,76561198146207066,Boombl4,Natus Vincere,NaN,25,12,18,0.67,6,2,0,0,0,1678,2037,8,67.1,295,62,0.21,0.89,76.0,6,0.5,3,5,34,15,25,6,13,19,30,38.0,0,1


## KAST Visualization

### By Team

In [10]:
custom_theta = ["Kills", "Assists", "Shots", "Trades"]

fig = go.Figure()

team_names = df.teamName.unique().tolist()
fig.add_trace(go.Scatterpolar(
    r= ((df.query(f"teamName=='{team_names[0]}'")[cats].sum(axis=0) /df[cats].sum(axis=0) ) * 100).astype(int),
    theta=custom_theta,
    fill="toself",
    name=f"{team_names[0]}"
))

# fig.add_trace(go.Scatterpolar(
#     r= ((df.query(f"teamName=='{team_names[1]}'")[cats].sum(axis=0) /df[cats].sum(axis=0) ) * 100).astype(int),

#     theta=custom_theta,
#     fill="toself",
#     name=f"{team_names[1]}"
# ))

fig.update_layout(
  polar=dict(
    radialaxis=dict(
      visible=True
    ),
  ),
  showlegend=True
)

fig.show()

# By player

In [24]:
df_players = ((df[cats].max(0)-df[cats])/(df[cats].max(0)-df[cats].min(0))*100).astype(int)

In [25]:
unique_players = df.playerName.unique().tolist()
df_players["playerName"] = unique_players

In [28]:
custom_theta = ["Kills", "Assists", "Shots", "Trades"]

fig = go.Figure()

unique_players = df.playerName.unique().tolist()

for player in unique_players[:5]:
    fig.add_trace(go.Scatterpolar(
        r= (df_players.query(f"playerName=='{player}'")[cats]).values.tolist()[0],

        theta=custom_theta,
        fill="toself",
        name=f"{player}"
    ))

    fig.update_layout(
    polar=dict(
        radialaxis=dict(
        visible=True
        ),
    ),
    showlegend=True
    )

fig.show()

fig = go.Figure()
for player in unique_players[5:]:
    fig.add_trace(go.Scatterpolar(
        r= (df_players.query(f"playerName=='{player}'")[cats]).values.tolist()[0],

        theta=custom_theta,
        fill="toself",
        name=f"{player}"
    ))

    fig.update_layout(
    polar=dict(
        radialaxis=dict(
        visible=True
        ),
    ),
    showlegend=True
    )

fig.show()

[]

In [20]:
data["gameRounds"][0].keys()

dict_keys(['roundNum', 'isWarmup', 'startTick', 'freezeTimeEndTick', 'endTick', 'endOfficialTick', 'bombPlantTick', 'tScore', 'ctScore', 'endTScore', 'endCTScore', 'ctTeam', 'tTeam', 'winningSide', 'winningTeam', 'losingTeam', 'roundEndReason', 'ctFreezeTimeEndEqVal', 'ctRoundStartEqVal', 'ctRoundSpendMoney', 'ctBuyType', 'tFreezeTimeEndEqVal', 'tRoundStartEqVal', 'tRoundSpendMoney', 'tBuyType', 'ctSide', 'tSide', 'kills', 'damages', 'grenades', 'bombEvents', 'weaponFires', 'flashes', 'frames'])

In [ ]:
req_keys = ['tScore', 'ctScore',
            'winningSide', 'winningTeam',
            'ctRoundSpendMoney',"tRoundSpendMoney",
            'ctSide', 'tSide']

In [65]:
[np.array([gameround.get("roundNum"),
           gameround.get("winningTeam"),
           gameround.get("winningSide"),
           gameround.get("ctRoundSpendMoney"),
           gameround.get("tRoundSpendMoney"),
           ]) for gameround in data["gameRounds"]]

[array(['1', 'OGEsports', 'CT', '3400'], dtype='<U21'),
 array(['2', 'Natus Vincere', 'T', '17950'], dtype='<U21'),
 array(['3', 'Natus Vincere', 'T', '8050'], dtype='<U21'),
 array(['4', 'Natus Vincere', 'T', '300'], dtype='<U21'),
 array(['5', 'OGEsports', 'CT', '21600'], dtype='<U21'),
 array(['6', 'OGEsports', 'CT', '16550'], dtype='<U21'),
 array(['7', 'Natus Vincere', 'T', '10800'], dtype='<U21'),
 array(['8', 'OGEsports', 'CT', '15950'], dtype='<U21'),
 array(['9', 'OGEsports', 'CT', '8800'], dtype='<U21'),
 array(['10', 'OGEsports', 'CT', '23100'], dtype='<U21'),
 array(['11', 'OGEsports', 'CT', '5100'], dtype='<U21'),
 array(['12', 'OGEsports', 'CT', '10200'], dtype='<U21'),
 array(['13', 'OGEsports', 'CT', '17150'], dtype='<U21'),
 array(['14', 'Natus Vincere', 'T', '23300'], dtype='<U21'),
 array(['15', 'Natus Vincere', 'T', '20300'], dtype='<U21'),
 array(['16', 'Natus Vincere', 'CT', '3400'], dtype='<U21'),
 array(['17', 'OGEsports', 'T', '19050'], dtype='<U21'),
 array(['

In [29]:
data["gameRounds"][0]["winningTeam"]

'OGEsports'

In [66]:
get_player_names = lambda x: df.playerName[df.teamName==x].unique().tolist()

In [14]:
for k in data["gameRounds"][0].keys():
    print(k)

roundNum
isWarmup
startTick
freezeTimeEndTick
endTick
endOfficialTick
bombPlantTick
tScore
ctScore
endTScore
endCTScore
ctTeam
tTeam
winningSide
winningTeam
losingTeam
roundEndReason
ctFreezeTimeEndEqVal
ctRoundStartEqVal
ctRoundSpendMoney
ctBuyType
tFreezeTimeEndEqVal
tRoundStartEqVal
tRoundSpendMoney
tBuyType
ctSide
tSide
kills
damages
grenades
bombEvents
weaponFires
flashes
frames


# Unsupervised

In [67]:
kills_df = pd.concat([pd.DataFrame(data["gameRounds"][i]["kills"][:]) for i in range(len(data["gameRounds"]))], axis=0)

In [68]:
cols = [
    "attackerName","attackerTeam","attackerSide",
    "attackerX", "attackerY", "attackerZ",
    "victimX", "victimY", "victimZ",
    "distance", "weapon", "weaponClass"
    ]

In [69]:
kills_df = kills_df.filter(items=cols)

In [70]:
axis_cols = [
    "attackerX", "attackerY",
    "victimX", "victimY"
]

In [71]:
mapfunc = lambda map,pos,axis: position_transform(map_name=map, position=pos, axis=axis)

In [72]:
for col in axis_cols:
    kills_df[col] = mapfunc(map=data["mapName"], pos=kills_df[col].values, axis=col[-1].lower())

In [73]:
def plot_map2(
    map_name: str = "de_dust2", map_type: str = "original", *, dark: bool = False
):
    """Plots a blank map.

    Args:
        map_name (str, optional): Map to search. Defaults to "de_dust2"
        map_type (str, optional): "original" or "simpleradar". Defaults to "original"
        dark (bool, optional): Only for use with map_type="simpleradar".
            Indicates if you want to use the SimpleRadar dark map type
            Defaults to False

    Returns:
        matplotlib fig and ax
    """
    base_path = os.path.join(os.path.dirname(awpydata.__file__), f"""../data/map/{map_name}""")

    if map_type == "original":
        map_bg = imread(f"{base_path}.png")
        if map_name in MAP_DATA and "z_cutoff" in MAP_DATA[map_name]:
            map_bg_lower = imread(f"{base_path}_lower.png")
            map_bg = np.concatenate([map_bg, map_bg_lower])
    else:
        try:
            col = "dark" if dark else "light"
            map_bg = imread(f"{base_path}_{col}.png")
            if map_name in MAP_DATA and "z_cutoff" in MAP_DATA[map_name]:
                map_bg_lower = imread(f"{base_path}_lower_{col}.png")
                map_bg = np.concatenate([map_bg, map_bg_lower])
        except FileNotFoundError:
            map_bg = imread(f"{base_path}.png")
            if map_name in MAP_DATA and "z_cutoff" in MAP_DATA[map_name]:
                map_bg_lower = imread(f"{base_path}_lower.png")
                map_bg = np.concatenate([map_bg, map_bg_lower])

    fig = px.imshow(map_bg,
                    width=1000,height=1000)

    fig.update_xaxes(showticklabels=False)
    fig.update_yaxes(showticklabels=False)

    # fig.show()
    return fig

    # figure, axes = plt.subplots()
    # axes.imshow(map_bg, zorder=0)
    # return figure, axes

In [74]:
from sklearn.neighbors import (
    KernelDensity
)

In [135]:
kills_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 162 entries, 0 to 8
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   attackerName  162 non-null    object 
 1   attackerTeam  162 non-null    object 
 2   attackerSide  162 non-null    object 
 3   attackerX     162 non-null    float64
 4   attackerY     162 non-null    float64
 5   attackerZ     162 non-null    float64
 6   victimX       162 non-null    float64
 7   victimY       162 non-null    float64
 8   victimZ       162 non-null    float64
 9   distance      162 non-null    float64
 10  weapon        162 non-null    object 
 11  weaponClass   162 non-null    object 
dtypes: float64(7), object(5)
memory usage: 16.5+ KB


In [76]:
kills_df = kills_df.dropna()

In [118]:
sides = ["CT", "T"]
X = kills_df.query("attackerSide==@sides[1]")[["attackerX", "attackerY"]]

y = kills_df.query("attackerSide==@sides[1]").weaponClass.apply(lambda x: 0 if x=="Grenade" else 1)

In [113]:
y.value_counts()

1    78
0     3
Name: weaponClass, dtype: int64

In [94]:
kde = KernelDensity(bandwidth=10,
                    metric="euclidean",
                    kernel="cosine")

In [95]:
kde.fit(X=X, y=y)

KernelDensity(bandwidth=10, kernel='cosine')

In [96]:
z = np.exp(kde.score_samples(X))

In [114]:
expit(np.exp(kde.score_samples(X[y==1])))

array([0.50000772, 0.50000772, 0.50000772, 0.50000772, 0.50000772,
       0.50000772, 0.50000772, 0.50000772, 0.50000772, 0.50000772,
       0.50000772, 0.50000772, 0.50000947, 0.50000772, 0.50000772,
       0.50000772, 0.50000772, 0.50000772, 0.50000772, 0.50000772,
       0.50000772, 0.50000772, 0.50000772, 0.50001143, 0.50001143,
       0.50000772, 0.50000772, 0.50000772, 0.50000772, 0.50000772,
       0.50000772, 0.50000772, 0.50000772, 0.50000772, 0.50000772,
       0.50000772, 0.50000772, 0.50000772, 0.50000772, 0.50000772,
       0.50001408, 0.50000772, 0.50000772, 0.50000772, 0.50000903,
       0.50000903, 0.50000772, 0.50000772, 0.50000772, 0.50001181,
       0.50001181, 0.50000772, 0.50001408, 0.50000772, 0.50000772,
       0.50000772, 0.50001174, 0.50000772, 0.50000947, 0.50000772,
       0.50000772, 0.50000772, 0.50001174, 0.50000772, 0.50000772,
       0.50000804, 0.50000772, 0.50000772, 0.50000772, 0.50000772,
       0.50000804, 0.50000772, 0.50000772, 0.50000772, 0.50000

In [97]:
from scipy.special import expit

In [98]:
expit(z)

array([0.50000772, 0.50000772, 0.50000772, 0.50000772, 0.50000772,
       0.50000772, 0.50000772, 0.50000772, 0.50000772, 0.50000772,
       0.50000772, 0.50000772, 0.50000947, 0.50000772, 0.50000772,
       0.50000772, 0.50000772, 0.50000772, 0.50000772, 0.50000772,
       0.50000772, 0.50000772, 0.50000772, 0.50001143, 0.50001143,
       0.50000772, 0.50000772, 0.50000772, 0.50000772, 0.50000772,
       0.50000772, 0.50000772, 0.50000772, 0.50000772, 0.50000772,
       0.50000772, 0.50000772, 0.50000772, 0.50000772, 0.50000772,
       0.50000772, 0.50000772, 0.50000772, 0.50001408, 0.50000772,
       0.50000772, 0.50000772, 0.50000903, 0.50000903, 0.50000772,
       0.50000772, 0.50000772, 0.50001181, 0.50001181, 0.50000772,
       0.50001408, 0.50000772, 0.50000772, 0.50000772, 0.50001174,
       0.50000772, 0.50000947, 0.50000772, 0.50000772, 0.50000772,
       0.50001174, 0.50000772, 0.50000772, 0.50000804, 0.50000772,
       0.50000772, 0.50000772, 0.50000772, 0.50000804, 0.50000

In [99]:
def plot_map2(
    map_name: str = "de_dust2", map_type: str = "original", *, dark: bool = False
):
    """Plots a blank map.

    Args:
        map_name (str, optional): Map to search. Defaults to "de_dust2"
        map_type (str, optional): "original" or "simpleradar". Defaults to "original"
        dark (bool, optional): Only for use with map_type="simpleradar".
            Indicates if you want to use the SimpleRadar dark map type
            Defaults to False

    Returns:
        matplotlib fig and ax
    """
    base_path = os.path.join(os.path.dirname(awpydata.__file__), f"""../data/map/{map_name}""")

    if map_type == "original":
        map_bg = imread(f"{base_path}.png")
        if map_name in MAP_DATA and "z_cutoff" in MAP_DATA[map_name]:
            map_bg_lower = imread(f"{base_path}_lower.png")
            map_bg = np.concatenate([map_bg, map_bg_lower])
    else:
        try:
            col = "dark" if dark else "light"
            map_bg = imread(f"{base_path}_{col}.png")
            if map_name in MAP_DATA and "z_cutoff" in MAP_DATA[map_name]:
                map_bg_lower = imread(f"{base_path}_lower_{col}.png")
                map_bg = np.concatenate([map_bg, map_bg_lower])
        except FileNotFoundError:
            map_bg = imread(f"{base_path}.png")
            if map_name in MAP_DATA and "z_cutoff" in MAP_DATA[map_name]:
                map_bg_lower = imread(f"{base_path}_lower.png")
                map_bg = np.concatenate([map_bg, map_bg_lower])

    fig = px.imshow(map_bg,
                    width=1000,height=1000)

    fig.update_xaxes(showticklabels=False)
    fig.update_yaxes(showticklabels=False)

    # fig.show()
    return fig, map_bg

    # figure, axes = plt.subplots()
    # axes.imshow(map_bg, zorder=0)
    # return figure, axes

array([3.08641975e-05, 3.08641975e-05, 3.08641975e-05])

In [235]:
fig, map_bg = plot_map2()

In [233]:
mask = np.all(map_bg == [0,0,0], axis = -1)
idx = np.argwhere(mask)
black_only = map_bg[mask] = [0,0,0]

In [236]:
fig.add_trace(
    go.Histogram2d(
    x= X.attackerX.values,
    # nbinsx=5,
    # nbinsy=5,
    y = X.attackerY.values,
    colorscale = [[0, 'rgb(0,0,0)'], [0.25, 'rgb(10,136,186)'], [0.5, 'rgb(242,211,56)'], [0.75, 'rgb(242,143,56)'], [1, 'rgb(217,30,30)']],
    opacity=0.25,
    showscale=True,
    histfunc="sum",
    histnorm="percent"
)
)
fig.add_trace(go.Image(z=black_only), )

In [237]:
fig.add_trace(go.Image(z=black_only))

In [159]:
fig.add_trace(
    go.Histogram2dContour(
    x= X.attackerX.values,
    ncontours=10,
    y = X.attackerY.values,
    colorscale="Jet",
    opacity=0.25,
    showscale=True
)
)

In [115]:
fig.add_trace(
    go.Contour(
    x= X.attackerX.values,
    y = X.attackerY.values,
    z = z,
    colorscale="RdBu_r",
    opacity=0.75,
    showscale=False
)
)

In [169]:

from sklearn.datasets import fetch_species_distributions
from sklearn.neighbors import KernelDensity

In [170]:
def construct_grids(batch):
    """Construct the map grid from the batch object

    Parameters
    ----------
    batch : Batch object
        The object returned by :func:`fetch_species_distributions`

    Returns
    -------
    (xgrid, ygrid) : 1-D arrays
        The grid corresponding to the values in batch.coverages
    """
    # x,y coordinates for corner cells
    xmin = batch.x_left_lower_corner + batch.grid_size
    xmax = xmin + (batch.Nx * batch.grid_size)
    ymin = batch.y_left_lower_corner + batch.grid_size
    ymax = ymin + (batch.Ny * batch.grid_size)

    # x coordinates of the grid cells
    xgrid = np.arange(xmin, xmax, batch.grid_size)
    # y coordinates of the grid cells
    ygrid = np.arange(ymin, ymax, batch.grid_size)

    return (xgrid, ygrid)

In [239]:
# data = fetch_species_distributions()
species_names = ["Bradypus Variegatus", "Microryzomys Minutus"]

Xtrain = np.vstack([data["train"]["dd lat"], data["train"]["dd long"]]).T
ytrain = np.array(
    [d.decode("ascii").startswith("micro") for d in data["train"]["species"]],
    dtype="int",
)



array([-56.  , -55.95, -55.9 , ...,  23.45,  23.5 ,  23.55])

In [235]:
Xtrain *= np.pi / 180.0  # Convert lat/long to radians

# Set up the data grid for the contour plot
xgrid, ygrid = construct_grids(data)
X, Y = np.meshgrid(xgrid[::5], ygrid[::5][::-1])
land_reference = data.coverages[6][::5, ::5]
land_mask = (land_reference > -9999).ravel()

xy = np.vstack([Y.ravel(), X.ravel()]).T
xy = xy[land_mask]
xy *= np.pi / 180.0


In [238]:
del xy

In [236]:
xy

array([[ 0.41015237, -1.35699349],
       [ 0.41015237, -1.35263017],
       [ 0.41015237, -1.33081355],
       ...,
       [-0.97738438, -1.17809725],
       [-0.97738438, -1.17373392],
       [-0.97738438, -1.1693706 ]])

In [181]:
np.vstack([data["train"]["dd lat"], data["train"]["dd long"]]).T

array([[-17.85   , -64.7    ],
       [-16.3333 , -67.8333 ],
       [-16.3    , -67.8833 ],
       ...,
       [ -0.3    , -78.4667 ],
       [  0.6    , -77.8167 ],
       [  4.96667, -75.3833 ]], dtype=float32)